Importa modulos

In [31]:
import pandas as pd
import matplotlib
import geopandas as gpd
from shapely.geometry import Point, Polygon
import shapely
import matplotlib.pyplot as plt
import json
import numpy as np
import pyproj
import requests
from bs4 import BeautifulSoup as bs
%matplotlib inline

Distancia lineal al subte

Funciones a usar

In [2]:
def punto_a_lista(punto): # Transforma objeto punto (shapely.geometry,Point) a lista de coordenadas
    lista=[]
    punto=str(punto)[7:-1]
    lista.append(float(punto[:punto.find(" ")]))
    lista.append(float(punto[punto.find(" ")+1:]))
    return lista

def distancia_puntos(punto1,punto2): # Calcula la distancia entre objetos punto (shapely.geometry,Point) en metros (proyeccion WGS84)
    punto1,punto2 = punto_a_lista(punto1),punto_a_lista(punto2)
    lat1,long1 = (punto1[1],punto1[0])
    lat2,long2 = (punto2[1],punto2[0])
    geod = pyproj.Geod(ellps="WGS84")
    angulo1,angulo2,distancia = geod.inv(long1, lat1, long2, lat2)
    return distancia

def make_point(df): # Genera objeto punto (shapely.geometry,Point) de las columnas "longitud"y "latitud"
    return Point(df["longitud"],df["latitud"])

In [3]:
dfFiltrado=pd.read_csv("dfFiltrado.csv") 

In [4]:
df=dfFiltrado[["longitud","latitud"]].dropna()

In [5]:
df.head()

,longitud,latitud
0,-58.401586,-34.599427
1,-58.390053,-34.588349
2,-58.412713,-34.595598
3,-58.404228,-34.583782
4,-58.386477,-34.590333


In [6]:
df["geom"]=df.apply(make_point, axis = 1)

In [7]:
gdfFiltrado = gpd.GeoDataFrame(df["geom"], geometry="geom")

In [8]:
subte = gpd.read_file("estaciones_de_subte.shp",crs="estaciones_de_subte.prj")
subte["geometry"]=subte["geometry"].to_crs({'init': 'epsg:4326'})
subte.head()

,ID,ESTACION,LINEA,geometry
0,1.0,CASEROS,H,POINT (-58.39833005827354 -34.63619629114226)
1,2.0,INCLAN,H,POINT (-58.40037199947714 -34.62982192437689)
2,3.0,HUMBERTO 1°,H,POINT (-58.40172516912597 -34.62353873781329)
3,4.0,VENEZUELA,H,POINT (-58.40413412280196 -34.61568876109078)
4,5.0,ONCE - 30 DE DICIEMBRE,H,POINT (-58.40543878453062 -34.60938200117359)


In [9]:
mini = []  # Calcula la distancia al subte y define que estación con la ubicación
minie = []
minin = []
for i in gdfFiltrado.index:
    mini.append(100000000)
    minie.append("")
    minin.append("")
for i in subte.index:
    contador=0
    for j in gdfFiltrado["geom"]:
        dist=distancia_puntos(subte["geometry"][i],j)
        if dist<mini[contador]:
            mini[contador] = dist
            minie[contador] = subte["geometry"][i]
            minin[contador] = subte["ESTACION"][i]
        contador=contador+1

In [10]:
gdfFiltrado["estacion_mas_cercanan"]=minin
gdfFiltrado["dist_estacion_mas_cercana"]=mini
gdfFiltrado["estacion_mas_cercanap"]=minie

In [11]:
lista = gdfFiltrado["estacion_mas_cercanan"].value_counts()[:12].index #filtra para sacar los outliers 

In [12]:
listat=[] 
for i in gdfFiltrado["estacion_mas_cercanan"]:
    if i in lista:
        listat.append(True)
    else:
        listat.append(False)
gdfFiltrado["mascara"]=listat

In [13]:
gdfFiltrado=pd.DataFrame(gdfFiltrado[gdfFiltrado["mascara"]==True].iloc[:,:-1])#saca los outliers 

In [14]:
gdfFiltrado=pd.DataFrame(gdfFiltrado) 
gdfFiltrado

,geom,estacion_mas_cercanan,dist_estacion_mas_cercana,estacion_mas_cercanap
0,POINT (-58.4015856 -34.5994274),CÓRDOBA,152.682471,POINT (-58.40312406017082 -34.59890209976835)
1,POINT (-58.3900529339 -34.5883487899),LAS HERAS,604.317026,POINT (-58.39661845686523 -34.58790875401352)
2,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101)
3,POINT (-58.40422750000001 -34.5837821),BULNES,804.959847,POINT (-58.41069625970212 -34.5886843671808)
4,POINT (-58.38647710000001 -34.590333),LAS HERAS,968.477120,POINT (-58.39661845686523 -34.58790875401352)
5,POINT (-58.40062320000001 -34.5910354),PUEYRREDON,439.124379,POINT (-58.40179798390445 -34.59487279229789)
6,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101)
7,POINT (-58.39736360000001 -34.5895459),LAS HERAS,194.053551,POINT (-58.39661845686523 -34.58790875401352)
8,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101)
9,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101)


In [15]:
#dfFiltrado = dfFiltrado.merge(gdfFiltrado,left_index=True, right_index=True) # junta los dataframe

Distancia al subte por isocronas ORS

Bajar poligonos isocronos

Funciones a usar

In [95]:
def mapa_isocrono(locations,rango,intervalo): # hace el requests (Para usar se necesita APIkey de http://openrouteservice.org en archivo apiORS.txt)
    api = str(open("apiORS.txt","r").read())
    url = "https://api.openrouteservice.org/isochrones?api_key=" +str(api)+"&profile=foot-walking&locations="+str(locations)+"&profile=foot-walking&range_type=distance&range="+str(rango)+"&interval="+str(intervalo)+"&location_type=start&intersections=false&id=h"
    response = requests.get(url)
    return response

def listadea5(puntos,cantidad): # parte los puntos en 5 para hace la request
    puntos2=""
    lista=[]
    puntos = puntos.replace("%7C","|")
    contador=0
    print(puntos,cantidad)
    for i in range(0,cantidad,1):
        puntos2 = puntos2 + puntos[:puntos.find("|")+1]
        puntos = puntos[puntos.find("|")+1:]
        contador = contador +1
        if contador == 5:
            contador = 0
            lista.append(puntos2[:-1].replace("|","%7C"))
            puntos2 = ""
    lista.append(puntos2.replace("|","%7C"))
    lista.append(puntos.replace("|","%7C"))
    return lista

def isocronoORS(gs,maximo,paso): #genera los poligonos isocronos
    lista = []
    for i in gs.index:
        lista.append(str(gs[i])[7:-1].replace(" ",","))  
    puntos = ""
    contador=0 
    for i in lista:
        puntos = puntos + lista[contador] + "|"
        contador = contador + 1
    puntos = puntos[:(len(puntos)-1)]
    puntos = puntos.replace(",","%2C")
    agrupadosdea5 = listadea5(puntos,len(gs.index))
    isocrono = gpd.GeoSeries()
    contador = 0
    for i in agrupadosdea5:
        var = mapa_isocrono(agrupadosdea5[contador],maximo,paso)  # 2do parametro amplitud max 3er parametro intervalo
        isocronores = json.loads(str(var.text))
        del isocronores["info"]
        lista=[]
        for i in isocronores ['features']:
            lista.append(Polygon(i['geometry']['coordinates'][0]))
        isocrono = isocrono.append(gpd.GeoDataFrame(geometry=lista))
        contador = contador + 1
    return isocrono["geometry"]

In [17]:
gs= subte["geometry"]

In [18]:
iso500_100 = isocronoORS(gs,500,100)
iso1000_100 = isocronoORS(gs,1000,100)

Ver puntos en isocronos

In [19]:
def buscarenisocrono(puntos,isocrono,partes): # ubica los puntos en isocronos
    isocrono = gpd.GeoDataFrame(isocrono)
    isocrono["circulo"]=(pd.Series(isocrono.index)%partes) + 1
    lista=[]
    for i in puntos:
        contador=0
        variable = partes + 1
        for j in isocrono.iloc[:,0]:
            if shapely.geometry.Polygon.contains(j,i) == True:
                if (contador%partes)+1 < variable:
                    variable = (contador%partes)+1
            contador=contador+1
        lista.append(variable)
    return lista

In [24]:
distiso500_100=buscarenisocrono(dfFiltrado["geom"],iso500_100,5)
distiso1000_100=buscarenisocrono(dfFiltrado["geom"],iso1000_100,10)

In [25]:
dfFiltrado["distiso500_100"]=distiso500_100
dfFiltrado["distiso1000_100"]=distiso1000_100


In [26]:
dfFiltrado

,Unnamed: 0,barrio,geonames_id,latitud,longitud,precio_usd,superficie_m2,superficie_cubierta,precio_usd_m2,piso,...,titulo,imagen,frente,luminoso,geom,estacion_mas_cercanan,dist_estacion_mas_cercana,estacion_mas_cercanap,distiso500_100,distiso1000_100
0,200,Recoleta,3429595.0,-34.599427,-58.401586,380000.0,196.0,196.0,1938.775510,NaN,...,Semipiso. Living. Comedor. 4 dormitorio(s) 1 e...,https://thumbs4.properati.com/3/Iokg3DyiJu3v_e...,1,1,POINT (-58.4015856 -34.5994274),CÓRDOBA,152.682471,POINT (-58.40312406017082 -34.59890209976835),1,1
1,227,Recoleta,3429595.0,-34.588349,-58.390053,620000.0,191.0,181.0,3246.073298,NaN,...,Recoleta Quintana Ancha 5 Amb,https://thumbs4.properati.com/2/AZGb5-RYl59wVS...,1,0,POINT (-58.3900529339 -34.5883487899),LAS HERAS,604.317026,POINT (-58.39661845686523 -34.58790875401352),6,7
2,255,Recoleta,3429595.0,-34.595598,-58.412713,120000.0,33.0,33.0,3636.363636,NaN,...,Sanchez de Bustamante 1300-Recoleta,https://thumbs4.properati.com/0/FkdP8ymn9D3IuB...,0,0,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8
3,262,Barrio Norte,3436109.0,-34.583782,-58.404228,98000.0,37.0,37.0,2648.648649,NaN,...,Departamento. Living/Comedor. 2 dormitorio(s)....,https://thumbs4.properati.com/3/uPErfXjjHg3fyN...,0,0,POINT (-58.40422750000001 -34.5837821),BULNES,804.959847,POINT (-58.41069625970212 -34.5886843671808),6,9
4,412,Recoleta,3429595.0,-34.590333,-58.386477,690000.0,235.0,235.0,2936.170213,NaN,...,Montevideo 1700 entre Alvear y Quintana,https://thumbs4.properati.com/7/mzfxgkFbcL6syg...,1,0,POINT (-58.38647710000001 -34.590333),LAS HERAS,968.477120,POINT (-58.39661845686523 -34.58790875401352),6,11
5,422,Recoleta,3429595.0,-34.591035,-58.400623,378000.0,140.0,136.0,2700.000000,NaN,...,Departamento venta,https://thumbs4.properati.com/2/-FUozvmum1Wy4n...,0,0,POINT (-58.40062320000001 -34.5910354),PUEYRREDON,439.124379,POINT (-58.40179798390445 -34.59487279229789),3,3
6,636,Recoleta,3429595.0,-34.595598,-58.412713,167000.0,43.0,43.0,3883.720930,NaN,...,sanchez de bustamante 1300-Recoleta,https://thumbs4.properati.com/3/f4i4a01xidFYXT...,0,0,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8
7,641,Recoleta,3429595.0,-34.589546,-58.397364,105000.0,28.0,28.0,3750.000000,NaN,...,Sanchez de Bustamante 1300-Recoleta,https://thumbs4.properati.com/5/VKvguEDGILMpi7...,0,0,POINT (-58.39736360000001 -34.5895459),LAS HERAS,194.053551,POINT (-58.39661845686523 -34.58790875401352),1,1
8,643,Recoleta,3429595.0,-34.595598,-58.412713,130000.0,37.0,37.0,3513.513514,NaN,...,Sanchez de bustamante 1300-Recoleta,https://thumbs4.properati.com/1/oscB3IFHII4CeV...,0,0,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8
9,645,Recoleta,3429595.0,-34.595598,-58.412713,235000.0,61.0,61.0,3852.459016,NaN,...,Sanchez de Bustamante -Recoleta,https://thumbs4.properati.com/9/gcLXgXwLWnrN3y...,0,0,POINT (-58.4127129 -34.5955984),AGÜERO,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8


In [27]:
dfFiltrado.to_csv("dfFiltradoconsubte.csv")

Puntos importantes

In [41]:
def geo(direccion):
    api = str(open("apiGOOGLEMAPS.txt","r").read())
    url = "https://maps.googleapis.com/maps/api/geocode/xml?address=" + str(direccion) +"&key=" + str(api)
    response = requests.get(url)
    soup=bs(response.text)
    return(str(soup.location.lat.string)+","+str(soup.location.lng.string))

In [135]:
lista =["av. Callao y Av. Alvear","Plaza Gelly y Obes", "Plaza V. López y Planes"]

In [136]:
cord=[]
df = pd.DataFrame
for i in lista:
    cord.append(geo(i)

C:\Users\Administrador\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Administrador\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


1
1
1


In [138]:
df = pd.DataFrame(lista)
df["lista"]=lista
lista=[]
for i in cord:
    lista.append(i[i.find(",")+1:-1]+" "+i[:i.find(",")])
df["cord"]=lista
df["cord"] = "POINT ("+df["cord"]+")"

In [124]:
isopuntosimp2000_200 = isocronoORS(df["cord"],2000,200)

-58.388699%2C-34.5877748|-58.396749%2C-34.5857757|-58.389425%2C-34.5937279 3


In [126]:
distisopuntosimp2000_200=buscarenisocrono(dfFiltrado["geom"],isopuntosimp2000_200,10)

In [149]:
dfFiltrado["isopuntosimp2000_200"]=distisopuntosimp2000_200

In [141]:
mini = []  # Calcula la distancia al subte y define que estación con la ubicación
minie = []
minin = []
for i in gdfFiltrado.index:
    mini.append(100000000)
    minie.append("")
    minin.append("")
for i in df["cord"].index:
    contador=0
    for j in gdfFiltrado["geom"]:
        dist=distancia_puntos(df["cord"][i],j)
        if dist<mini[contador]:
            mini[contador] = dist
            minie[contador] = df["cord"][i]
            minin[contador] = df["lista"][i]
        contador=contador+1

In [142]:
gdfFiltrado["punto_mas_cercanan"]=minin
gdfFiltrado["dist_punto_mas_cercana"]=mini
gdfFiltrado["punto_mas_cercanap"]=minie

In [153]:
dfFiltrado = dfFiltrado.merge(gdfFiltrado,left_index=True, right_index=True)

In [160]:
dfFiltrado.to_csv("dfFiltradoconsubteypuntos.csv")

Api google

In [389]:
def listadea25(puntos,cantidad): # parte los puntos en 5 para hace la request
    puntos2=""
    lista=[]
    puntos = puntos.replace("%7C","|")
    contador=0
    for i in range(0,cantidad,1):
        puntos2 = puntos2 + puntos[:puntos.find("|")+1]
        puntos = puntos[puntos.find("|")+1:]
        contador = contador +1
        if contador == 25:
            contador = 0
            lista.append(puntos2[:-1].replace("|","%7C"))
            puntos2 = ""
    lista.append(puntos2.replace("|","%7C"))
    lista.append(puntos.replace("|","%7C"))
    return lista

def matriz_google(gs,destino): #genera los poligonos isocronos
    lista = []
    dfmat = pd.DataFrame()
    dfmat["puntos"] = gs
    for i in gs.index:
        string=str(gs[i])[7:-1]
        lista.append((string[string.find(" ")+1:]+","+string[:string.find(" ")+1]).replace(" ",""))  
    puntos = ""
    contador=0 
    for i in lista:
        puntos = puntos + lista[contador] + "|"
        contador = contador + 1
    puntos = puntos[:(len(puntos)-1)]
    puntos = puntos.replace(",","%2C")
    agrupadosdea25 = listadea25(puntos,len(gs.index))
    matiz = gpd.GeoSeries()
    contador = 0
    lista=[]
    for i in agrupadosdea25:
        string=str(destino)[7:-1]
        string=string[string.find(" ")+1:]+ "," +string[:string.find(" ")+1].replace(" ","")
        var = matriz_google_req(agrupadosdea25[contador],string.replace(",","%2C"))  # 2do parametro amplitud max 3er parametro intervalo
        dic = json.loads(str(var.text)) #agrega .text
        for i in np.arange(0,(len(dic['rows'])),1):
            lista.append(dic['rows'][int(i)]['elements'][0]['distance']['value'])
        contador = contador + 1   
    return lista

def matriz_google_req(origen,destino):
    api = str(open("apiGOOGLEMAPS.txt","r").read())
    url = str("https://maps.googleapis.com/maps/api/distancematrix/json?units=metric&mode=walking&origins=" + str(origen) + "&destinations=" + str(destino) + "&key=" + str(api))
    response = requests.get(url)
    return(response)

In [414]:
listagoo = []
for i in dfFiltrado["estacion_mas_cercanap_x"].value_counts()[:12].index:
    temporal = dfFiltrado["geom_x"][dfFiltrado["estacion_mas_cercanap_x"]==i]
    listagoo.append(matriz_google(temporal,i))

[866, 1304, 193, 1144, 797, 1055, 1043, 193, 193, 438, 1498, 653, 1083, 1044, 1044, 543, 1451, 1224, 193, 73, 193, 521, 1109, 723, 555, 193, 1083, 1106, 193, 195, 193, 193, 193, 193, 193, 1358, 193, 550, 1266, 418, 193, 422, 311, 976, 402, 1266, 760, 749, 236, 422, 396, 1418, 431, 431, 1184, 483, 199, 659, 906, 713, 799, 618, 1147, 255, 470, 174, 1023, 785, 1200, 250, 1099, 1147, 193, 193, 231, 621, 588, 523, 666, 556, 1299, 1070, 1067, 1073, 1070, 665, 1019, 570, 1078, 1075, 741, 893, 1023, 752, 900, 193, 193, 429, 429, 936, 1086, 286, 664, 1274, 558, 887, 193, 431, 282, 1118, 1226, 418, 193, 193, 1274, 936, 193, 193, 1299, 395, 662, 193, 1121, 657, 774, 193, 1762, 193, 575, 129, 257, 129, 193, 311, 149, 415, 415, 424, 923, 193, 264, 741, 193, 193, 193, 705, 446, 438, 193, 652, 1166, 239, 239, 193, 193, 830, 979, 527, 155, 193, 599, 276, 748, 193, 193, 193, 1152, 417, 193, 946, 193, 193, 193, 193, 1123, 1142, 1263, 193, 193, 81, 770, 77, 193, 871, 740, 454, 193, 193, 1043, 1117, 281, 

In [500]:
lista = []
lista2 = []
lista3 = []
for i in listagoo:
    for j in i.index:
        lista.append(i['puntos'][j])
        lista2.append(i["distancias"][j])
        lista3.append(j)

In [504]:
df=pd.DataFrame(data=lista2, index=lista3,columns=["subtegoo"])

In [507]:
dfFiltrado = dfFiltrado.merge(df,how="outer",left_index=True,right_index=True)

In [508]:
dfFiltrado

,Unnamed: 0,barrio,geonames_id,latitud,longitud,precio_usd,superficie_m2,superficie_cubierta,precio_usd_m2,piso,...,dist_estacion_mas_cercana_x,estacion_mas_cercanap_x,distiso500_100,distiso1000_100,isopuntosimp2000_200,punto_mas_cercanan,dist_punto_mas_cercana,punto_mas_cercanap,subtegoo_x,subtegoo_y
0,200,Recoleta,3429595.0,-34.599427,-58.401586,380000.0,196.0,196.0,1938.775510,NaN,...,152.682471,POINT (-58.40312406017082 -34.59890209976835),1,1,8,Plaza V. López y Planes,1282.258353,POINT (-58.389425 -34.5937279),167.0,167.0
1,227,Recoleta,3429595.0,-34.588349,-58.390053,620000.0,191.0,181.0,3246.073298,NaN,...,604.317026,POINT (-58.39661845686523 -34.58790875401352),6,7,1,av. Callao y Av. Alvear,139.583826,POINT (-58.388699 -34.5877748),866.0,866.0
2,255,Recoleta,3429595.0,-34.595598,-58.412713,120000.0,33.0,33.0,3636.363636,NaN,...,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8,10,Plaza Gelly y Obes,1825.438779,POINT (-58.396749 -34.5857757),909.0,909.0
3,262,Barrio Norte,3436109.0,-34.583782,-58.404228,98000.0,37.0,37.0,2648.648649,NaN,...,804.959847,POINT (-58.41069625970212 -34.5886843671808),6,9,4,Plaza Gelly y Obes,720.890663,POINT (-58.396749 -34.5857757),947.0,947.0
4,412,Recoleta,3429595.0,-34.590333,-58.386477,690000.0,235.0,235.0,2936.170213,NaN,...,968.477120,POINT (-58.39661845686523 -34.58790875401352),6,11,2,av. Callao y Av. Alvear,349.410580,POINT (-58.388699 -34.5877748),1304.0,1304.0
5,422,Recoleta,3429595.0,-34.591035,-58.400623,378000.0,140.0,136.0,2700.000000,NaN,...,439.124379,POINT (-58.40179798390445 -34.59487279229789),3,3,4,Plaza Gelly y Obes,683.208175,POINT (-58.396749 -34.5857757),473.0,473.0
6,636,Recoleta,3429595.0,-34.595598,-58.412713,167000.0,43.0,43.0,3883.720930,NaN,...,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8,10,Plaza Gelly y Obes,1825.438779,POINT (-58.396749 -34.5857757),909.0,909.0
7,641,Recoleta,3429595.0,-34.589546,-58.397364,105000.0,28.0,28.0,3750.000000,NaN,...,194.053551,POINT (-58.39661845686523 -34.58790875401352),1,1,2,Plaza Gelly y Obes,422.023516,POINT (-58.396749 -34.5857757),193.0,193.0
8,643,Recoleta,3429595.0,-34.595598,-58.412713,130000.0,37.0,37.0,3513.513514,NaN,...,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8,10,Plaza Gelly y Obes,1825.438779,POINT (-58.396749 -34.5857757),909.0,909.0
9,645,Recoleta,3429595.0,-34.595598,-58.412713,235000.0,61.0,61.0,3852.459016,NaN,...,686.280142,POINT (-58.40656380624967 -34.5920750047101),6,8,10,Plaza Gelly y Obes,1825.438779,POINT (-58.396749 -34.5857757),909.0,909.0


In [509]:
dfFiltrado.to_csv("dfFiltradoconsubtecomplypuntos.csv")